# Simulación de un cubo

# Simulación de imágenes independientes

In [ ]:
import random 
import numpy as np

# Ésta función nos va a dar la intensidad de una fuente simulada a partir de el data_cube inicial,
# el valor máximo de las varianzas y el valor máximo de la intensidad de la fuente.


def mock_data( data_cube, sigma_max_x, sigma_max_y, Amax ):

    # Creamos una matriz M de ceros con el tamaño del cubo de datos:
    xmax = np.shape(data_cube)[0]
    ymax = np.shape(data_cube)[1]
    M = np.zeros((xmax, ymax))
    # Generamos aleatoriamente la posición media de la fuente en los ejes x, y dentro de la imagen, así como los valores máximos para la intensidad y las varianzas:
    A = np.random.rand()*Amax #Intensidad (almacenarla y que sea siempre mayor al ruido blanco añadido)
    sigmax = np.random.rand()*sigma_max_x
    sigmay = np.random.rand()*sigma_max_y #anchuras
    x_mid = random.randint(0, xmax)
    y_mid = random.randint(0, ymax)
    label_structure={"class":"galaxy", "x":x_mid, "y":y_mid,"width":sigmax, "height": sigmay}

    # Calculamos una distribución gaussiana para la fuente en los 3 ejes:
    x = (np.arange(0,xmax)-x_mid)**2./(2.*sigmax**2.)
    y = (np.arange(0,ymax)-y_mid)**2./(2.*sigmay**2.)

    X,Y = np.meshgrid(x,y)
    
    M = A*np.exp(-X-Y)
    
    return [M,label_structure]

In [ ]:
data=[]
n_images=100
for n in range(n_images):
  # definimos el tamaño de los ejes del cubo
  x_len = 512
  y_len = 512

  # creamos una matriz de (512,512) de ceros para almacenar los datos
  data_cube = np.zeros((x_len, y_len))

  # definimos las varianzas y la intensidad máxima de las fuentes.

  Amax = 1
  sigma_max_x = 10 
  sigma_max_y= 10  

  # Añadimos un ruido blanco a la imagen en todas las frecuencias
  Noise = np.reshape(np.random.normal(0, Amax/50, x_len*y_len), (x_len, y_len)) # Por qué justo entre 50?
  data_cube_noise = data_cube + Noise

  # Creamos N fuentes con esas características.

  N = 100 #El número de galaxias

  fuentes=[] #Almacenamos las fuentes para el label
  ax=[]

  boxes=[]

  for i in range(N):
      mock=mock_data(data_cube_noise, sigma_max_x, sigma_max_y, Amax)
      boxes.append(mock[1])
      data_cube_noise = data_cube_noise + mock[0]

  data.append([data_cube_noise, boxes])
  print(n)

In [ ]:
#Creamos una lista para almacenar los arrays binarios
binaries=[]

#Creamos una lista para almacenar los arrays de etiquetas
labels=[]

for k in range(len(data)):
  #Definimos el tamaño de la imagen
  x_len = 512
  y_len = 512
  #Inicializamos los arrays binarios
  binary=np.zeros((x_len, y_len))
  #Recorremos los boxes
  for box in data[k][1]:
    #Definimos los límites de la caja
    x_min=int(box['x']-box['width'])
    x_max=int(box['x']+box['width'])
    y_min=int(box['y']-box['height'])
    y_max=int(box['y']+box['height'])
    #Rellenamos los límites con etiquetas
    binary[x_min:x_max, y_min:y_max] = 1
  #Almacenamos los arrays binarios
  binaries.append(binary)
  #Añadimos las etiquetas
  labels.append(data[k][1])

#Creamos una lista con todos los datos
data_total=[binaries, labels]

In [ ]:
# Displaying the array
print('Array:\n', data)
file = open("galaxies.txt", "w+")
 
# Saving the array in a text file
content = str(data)
file.write(content)
file.close()
 
# Displaying the contents of the text file
file = open("galaxies.txt", "r")
content = file.read()
 
print("\nContent in galaxies.txt:\n", content)
file.close()

In [ ]:
from astropy.io import fits
hdulist = fits.HDUList() #instancia HDUList

for n in data:
  data_cube_noise=n[0]
  from astropy.io import fits
  hdulist.append(fits.PrimaryHDU(data_cube_noise.T))# agregamos nuestro cubo de datos
  
#Guardamos el cubo
try:
  hdulist.writeto('data_imagen_simulada.fits')
except Exception as e:
  import os
  os.remove('data_imagen_simulada.fits')
  hdulist.writeto('data_imagen_simulada.fits')

In [ ]:
hdulist.info()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(hdulist[0].data, cmap="gray")

# Modelo

In [ ]:
images=[]
label=[]
for n in range (n_images):
  images.append(hdulist[n].data)
  label.append(data_total[0][n])


In [ ]:
#Como cada imagen es una nueva simulación no necesitamos aleatorizar los datos

# Definir tamaño de los conjuntos de entrenamiento y prueba 
train_size = int(0.7 * len(images))
test_size = len(images) - train_size

# Separar los conjuntos de entrenamiento y prueba 
train_images = images[:train_size]
train_label = label[:train_size]
test_images = images[train_size:]
test_label = label[train_size:]

In [ ]:
np.shape(train_images)
np.shape(train_label)

In [ ]:
#Importar las librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout

#Definir modelo
model = tf.keras.Sequential()

#Agregar capas
model.add(Input(shape=(512, 512, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

#Compilar modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#Entrenar modelo
model.fit(x=train_images, 
          y=train_label, 
          epochs=10, 
          validation_data=(test_images, test_label))